In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
import numpy as np
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import matplotlib.pyplot as plt

In [2]:
#idd_file='/home/ict4bd_v2/Desktop/building/energyplus/EnergyPlus-9-0-1/Energy+.idd'
iddfile='/home/ict4bd_v2/Desktop/building/energyplus/EnergyPlus-9-0-1/Energy+.idd'
fname = '/home/ict4bd_v2/Desktop/project/on_double.idf'
weather='/home/ict4bd_v2/Desktop/project/ITA_Torino_160590_IWEC.epw'
IDF.setiddname(iddfile)
idf = IDF(fname,weather)

In [3]:
zones = idf.idfobjects["ZONE"]
total_area=0
for i in range(len(zones)):
    zone = zones[i]
    area = modeleditor.zonearea(idf, zone.Name)
    total_area=total_area+area

print("total one area = %s" % (total_area, ))

total one area = 1084.534275140951


In [4]:
samples_temp =[]
samples_temp.append({'Orientation': 270,
                     'Insulation Thickness': 0.35,#0.23
                     'Window to Wall Ratio': 0.15,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Orientation,Insulation Thickness,Window to Wall Ratio
0,270,0.35,0.15


In [5]:
building = ef.get_building(fname)
insulation = FieldSelector(class_name='Material', object_name='MW Glass Wool (rolls)_O.1445', 
                           field_name='Thickness')
insulationPR=Parameter(selector=insulation,name='Insulation Thickness')

window_to_wall = wwr(CategoryParameter([0.15])) 

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientationPR = Parameter(selector=orientation, value_descriptor=CategoryParameter(options=[270]),
                          name='Orientation')

parameters = [orientationPR , window_to_wall, insulationPR]
objectives = ['Electricity:Facility','DistrictHeating:Facility','DistrictCooling:Facility'] # these get made into `MeterReader` or `VariableReader`
problem=EPProblem(parameters, objectives) # problem = parameters + objectives

/home/ict4bd_v2/py3/lib/python3.8/site-packages/besos-1.7.0-py3.8.egg/besos/eppy_funcs.py:108: UserWarning: idd is already set to: /home/ict4bd_v2/Desktop/building/energyplus/EnergyPlus-9-0-1/Energy+.idd. It will NOT be changed to the default of: /usr/local/EnergyPlus-9-0-1/Energy+.idd.
  warnings.warn(
/home/ict4bd_v2/py3/lib/python3.8/site-packages/besos-1.7.0-py3.8.egg/besos/parameters.py:592: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15])
  warnings.warn(


In [7]:
evaluator = EvaluatorEP(problem, building, out_dir='sim_outputdir', err_dir='sim_outputdir' ,
                        epw=weather) # evaluator = problem + building

In [49]:
#simulation 2017
run_period = idf.idfobjects['RunPeriod'][0]
run_period.Begin_Year = 2017
run_period.End_Year = 2017
run_period
idf.idfobjects['RunPeriod'][0]


RunPeriod,
    lab_part1 (O1-O1:31-12),    !- Name
    1,                        !- Begin Month
    1,                        !- Begin Day of Month
    2017,                     !- Begin Year
    12,                       !- End Month
    31,                       !- End Day of Month
    2017,                     !- End Year
    ,                         !- Day of Week for Start Day
    No,                       !- Use Weather File Holidays and Special Days
    No,                       !- Use Weather File Daylight Saving Period
    Yes,                      !- Apply Weekend Holiday Rule
    Yes,                      !- Use Weather File Rain Indicators
    Yes,                      !- Use Weather File Snow Indicators
    ;                         !- Treat Weather as Actual

In [193]:
#hourly simulation
for i in range(len(idf.idfobjects['OUTPUT:VARIABLE'])):
    idf.idfobjects['OUTPUT:VARIABLE'][i].Reporting_Frequency='hourly'
for i in range(len(idf.idfobjects['OUTPUT:METER'])):
    idf.idfobjects['OUTPUT:METER'][i].Reporting_Frequency='hourly'
idf.idfobjects['OUTPUT:ENVIRONMENTALIMPACTFACTORS'][0].Reporting_Frequency='hourly'

In [104]:
#Now we run the evaluator with the given parameters
result = evaluator([270,0.15,0.35]) 
values = dict(zip(objectives, result))
for key, value in values.items():
    print(key, " :: ", "{0:.2f}".format(value/3.6e6), "kWh")

Electricity:Facility  ::  50169.43 kWh
DistrictHeating:Facility  ::  37120.58 kWh
DistrictCooling:Facility  ::  7722.42 kWh


In [127]:
idf.run(readvars=True,output_directory=u'simulation_output/output_EPPY_hourly',annual= True)
idf_data =pd.read_csv('/home/ict4bd_v2/Desktop/project/simulation_output/output_EPPY_hourly/eplusout.csv')


/home/ict4bd_v2/Desktop/building/energyplus/EnergyPlus-9-0-1/energyplus --weather /home/ict4bd_v2/Desktop/project/ITA_Torino_160590_IWEC.epw --output-directory /home/ict4bd_v2/Desktop/project/simulation_output/output_EPPY_hourly --annual --idd /home/ict4bd_v2/Desktop/building/energyplus/EnergyPlus-9-0-1/Energy+.idd --readvars /home/ict4bd_v2/Desktop/project/in.idf



In [129]:
#getting required columns 
columns=['Date/Time']
for i in idf_data.columns:
    if ('Zone Operative Temperature' in i or 'District' in i or 'Electricity:Facility' in i or 'Drybulb' in i) and ('Hourly' in i): 
        columns=columns+[i]
columns

['Date/Time',
 'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
 'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE4:Zone Operative Temperature [C](Hourly)',
 'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)',
 'Electricity:Facility [J](Hourly)',
 'DistrictCooling:Facility [J](Hourly)',
 'DistrictHeating:Facility [J](Hourly)']

In [130]:
df_columns = {'Date/Time':'Date_Time',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)':'t_in_ZONE3',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)':'t_in_ZONE1',
              'BLOCK1:ZONE4:Zone Operative Temperature [C](Hourly)':'t_in_ZONE4',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)':'t_in_ZONE2',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}


In [169]:
idf_data=idf_data[columns]
data = idf_data.rename(columns =df_columns)
data

,Date_Time,t_out,t_in_ZONE3,t_in_ZONE1,t_in_ZONE4,t_in_ZONE2,power_electricity,power_cooling,power_heating
0,01/01 01:00:00,0.925,13.881985,14.704189,13.783859,13.845197,2.198661e+06,0.0,0.000000e+00
1,01/01 02:00:00,0.650,13.407667,13.970457,13.264381,13.349967,2.198661e+06,0.0,0.000000e+00
2,01/01 03:00:00,0.850,13.093317,13.557124,12.962577,13.026059,2.198661e+06,0.0,7.990230e+05
3,01/01 04:00:00,0.975,12.840838,13.204021,12.814173,12.830734,2.198661e+06,0.0,4.177752e+06
4,01/01 05:00:00,1.000,12.716023,12.894248,12.702670,12.713656,2.198661e+06,0.0,7.533797e+06
...,...,...,...,...,...,...,...,...,...
8755,12/31 20:00:00,0.875,17.025859,16.643237,17.123165,17.075176,4.147716e+07,0.0,8.158903e+07
8756,12/31 21:00:00,0.575,17.038292,16.846181,17.126960,17.087322,4.147716e+07,0.0,7.950374e+07
8757,12/31 22:00:00,0.275,15.341103,16.972907,15.432559,15.400036,2.773038e+07,0.0,1.723098e+07
8758,12/31 23:00:00,0.200,14.379886,17.045810,14.412993,14.420377,2.773038e+07,0.0,1.627729e+07


In [170]:
data['t_in'] = data[['t_in_ZONE3','t_in_ZONE1','t_in_ZONE4', 't_in_ZONE2']].mean(axis=1)

In [171]:
data=data.drop(['t_in_ZONE3', 't_in_ZONE1', 't_in_ZONE4','t_in_ZONE2'], axis = 1)
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in
0,01/01 01:00:00,0.925,2.198661e+06,0.0,0.000000e+00,14.053807
1,01/01 02:00:00,0.650,2.198661e+06,0.0,0.000000e+00,13.498118
2,01/01 03:00:00,0.850,2.198661e+06,0.0,7.990230e+05,13.159769
3,01/01 04:00:00,0.975,2.198661e+06,0.0,4.177752e+06,12.922441
4,01/01 05:00:00,1.000,2.198661e+06,0.0,7.533797e+06,12.756649
...,...,...,...,...,...,...
8755,12/31 20:00:00,0.875,4.147716e+07,0.0,8.158903e+07,16.966859
8756,12/31 21:00:00,0.575,4.147716e+07,0.0,7.950374e+07,17.024689
8757,12/31 22:00:00,0.275,2.773038e+07,0.0,1.723098e+07,15.786651
8758,12/31 23:00:00,0.200,2.773038e+07,0.0,1.627729e+07,15.064766


In [172]:
data['temp_diff'] =data['t_in'] - data['t_out']
data['Date_Time'] = '2017/' + data['Date_Time'].str.strip()
data['Date_Time'] = data['Date_Time'].str.replace('24:00:00','23:59:59')
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in,temp_diff
0,2019/01/01 01:00:00,0.925,2.198661e+06,0.0,0.000000e+00,14.053807,13.128807
1,2019/01/01 02:00:00,0.650,2.198661e+06,0.0,0.000000e+00,13.498118,12.848118
2,2019/01/01 03:00:00,0.850,2.198661e+06,0.0,7.990230e+05,13.159769,12.309769
3,2019/01/01 04:00:00,0.975,2.198661e+06,0.0,4.177752e+06,12.922441,11.947441
4,2019/01/01 05:00:00,1.000,2.198661e+06,0.0,7.533797e+06,12.756649,11.756649
...,...,...,...,...,...,...,...
8755,2019/12/31 20:00:00,0.875,4.147716e+07,0.0,8.158903e+07,16.966859,16.091859
8756,2019/12/31 21:00:00,0.575,4.147716e+07,0.0,7.950374e+07,17.024689,16.449689
8757,2019/12/31 22:00:00,0.275,2.773038e+07,0.0,1.723098e+07,15.786651,15.511651
8758,2019/12/31 23:00:00,0.200,2.773038e+07,0.0,1.627729e+07,15.064766,14.864766


In [173]:
#converting from J to kWh
data['power_heating'] /= 3.6e6
data['power_cooling'] /= 3.6e6
data['power_electricity'] /= 3.6e6

# for i in range(1,len(data.columns)):
#     data.iloc[:,i] = data.iloc[:,i].apply(lambda x: round(x, 2))

In [174]:
data

,Date_Time,t_out,power_electricity,power_cooling,power_heating,t_in,temp_diff
0,2019/01/01 01:00:00,0.925,0.610739,0.0,0.000000,14.053807,13.128807
1,2019/01/01 02:00:00,0.650,0.610739,0.0,0.000000,13.498118,12.848118
2,2019/01/01 03:00:00,0.850,0.610739,0.0,0.221951,13.159769,12.309769
3,2019/01/01 04:00:00,0.975,0.610739,0.0,1.160487,12.922441,11.947441
4,2019/01/01 05:00:00,1.000,0.610739,0.0,2.092721,12.756649,11.756649
...,...,...,...,...,...,...,...
8755,2019/12/31 20:00:00,0.875,11.521435,0.0,22.663619,16.966859,16.091859
8756,2019/12/31 21:00:00,0.575,11.521435,0.0,22.084372,17.024689,16.449689
8757,2019/12/31 22:00:00,0.275,7.702884,0.0,4.786383,15.786651,15.511651
8758,2019/12/31 23:00:00,0.200,7.702884,0.0,4.521469,15.064766,14.864766


In [177]:
data['total_power'] = data['power_heating']+ data['power_cooling']
data = data[['Date_Time','t_in','t_out','temp_diff','power_heating','power_cooling','power_electricity','total_power']]

In [180]:
data.to_csv(path_or_buf='/home/ict4bd_v2/Desktop/project/simulation_output/simulation_data_hourly.csv',index=False)

In [181]:
data

,Date_Time,t_in,t_out,temp_diff,power_heating,power_cooling,power_electricity,total_power
0,2019/01/01 01:00:00,14.053807,0.925,13.128807,0.000000,0.0,0.610739,0.000000
1,2019/01/01 02:00:00,13.498118,0.650,12.848118,0.000000,0.0,0.610739,0.000000
2,2019/01/01 03:00:00,13.159769,0.850,12.309769,0.221951,0.0,0.610739,0.221951
3,2019/01/01 04:00:00,12.922441,0.975,11.947441,1.160487,0.0,0.610739,1.160487
4,2019/01/01 05:00:00,12.756649,1.000,11.756649,2.092721,0.0,0.610739,2.092721
...,...,...,...,...,...,...,...,...
8755,2019/12/31 20:00:00,16.966859,0.875,16.091859,22.663619,0.0,11.521435,22.663619
8756,2019/12/31 21:00:00,17.024689,0.575,16.449689,22.084372,0.0,11.521435,22.084372
8757,2019/12/31 22:00:00,15.786651,0.275,15.511651,4.786383,0.0,7.702884,4.786383
8758,2019/12/31 23:00:00,15.064766,0.200,14.864766,4.521469,0.0,7.702884,4.521469
